In [243]:
import requests
import pandas as pd
from collections import defaultdict
from itertools import combinations
import json
import numpy as np
import os

# Directory containing the JSON files
folder_path = r'\Users\Zachary\Documents\GitHub\5v5s\json_files'

# List to hold DataFrames
dataframes = []

# Loop through all files in the directory
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        # Read each JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
            # Convert JSON data to DataFrame
            df = pd.DataFrame(data)
            # Append DataFrame to list
            dataframes.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

In [244]:
hold = []
for i in combined_df['participants']:
    hold.append(pd.Series(i))

#transpose participant data and attach to unclean df
with pd.option_context("future.no_silent_downcasting", True):
    unclean_df = combined_df.merge(pd.concat(hold, axis = 1).T, left_index= True, right_index=True,how = 'inner').drop(columns= ['participants', 'TOTAL_PLAYER_SCORE','TOTAL_SCORE_RANK','OBJECTIVE_PLAYER_SCORE','PUUID']).fillna(0)

In [245]:
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}

unclean_df['NAME'] = unclean_df['NAME'].apply(lambda x:player_aliases.get(x,x)) 
#Change names so that player entries are combined

In [246]:
min_matches = 8

print(unclean_df['matchId'].nunique())
num_games = unclean_df[['matchId','NAME']].groupby('NAME').count()
name_l10 = num_games[num_games['matchId'] < min_matches] #get the name of each player with less than min_matches games

# num_games = num_games[num_games['matchId'] >= 10]
name_g10 = num_games[num_games['matchId'] >= min_matches] #get the name of each player with more than min_matches games

#add a boolean for True is player has less than min_matches and False otherwise
s_g10 = unclean_df.copy()
s_g10['NAME'] = np.isin(s_g10['NAME'],np.array(name_l10.index))
unclean_df['l10'] = s_g10['NAME']

# find all matches with no member with less than min_matches agmes
df_save_lmember = unclean_df[['l10','matchId','NAME']].groupby('matchId').sum().sort_values('l10')
matches_gmemberonly = df_save_lmember[df_save_lmember['l10'] < 1].index

#keep only match information for players with more than min_matches games
cleaned_df = unclean_df[np.isin(unclean_df['matchId'],matches_gmemberonly)]
cleaned_df

62


,matchId,gameDuration,gameVersion,ALL_IN_PINGS,ASSIST_ME_PINGS,ASSISTS,BAIT_PINGS,BARON_KILLS,BARRACKS_KILLED,BARRACKS_TAKEDOWNS,...,WARD_PLACED,WARD_PLACED_DETECTOR,WAS_AFK,WAS_AFK_AFTER_FAILED_SURRENDER,WAS_EARLY_SURRENDER_ACCOMPLICE,WAS_LEAVER,WAS_SURRENDER_DUE_TO_AFK,WIN,COMBAT_PLAYER_SCORE,l10
40,4523056888,1602221.0,12.23.483.5208,0,0,9,0,0,1,0,...,9,2,0,0,0,0,0,Win,0,False
41,4523056888,1602221.0,12.23.483.5208,0,0,12,0,0,0,0,...,16,1,0,0,0,0,0,Win,0,False
42,4523056888,1602221.0,12.23.483.5208,0,0,8,0,0,0,0,...,3,2,0,0,0,0,0,Win,0,False
43,4523056888,1602221.0,12.23.483.5208,0,0,8,0,1,0,0,...,11,2,0,0,0,0,0,Win,0,False
44,4523056888,1602221.0,12.23.483.5208,0,0,11,0,0,0,0,...,15,6,0,0,0,0,0,Win,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,5031483856,1970770.0,14.13.598.9749,0,0,3,0,0,0,0,...,9,0,0,0,0,0,0,Fail,0,False
616,5031483856,1970770.0,14.13.598.9749,0,1,7,0,0,0,0,...,12,1,0,0,0,0,0,Fail,0,False
617,5031483856,1970770.0,14.13.598.9749,0,0,10,0,0,0,0,...,26,2,0,0,0,0,0,Fail,0,False
618,5031483856,1970770.0,14.13.598.9749,0,1,8,0,0,0,0,...,12,1,0,0,0,0,0,Fail,0,False


In [247]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

[]

In [248]:
for i in df.columns:
    df[i] = pd.to_numeric(df[i], errors = 'ignore')


C:\Users\Zachary\AppData\Local\Temp\ipykernel_25356\1678985686.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[i] = pd.to_numeric(df[i], errors = 'ignore')


In [249]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

[]

In [250]:
# df[['NAME','CHAMPIONS_KILLED']].groupby('NAME').count()['CHAMPIONS_KILLED'].sort_values(ascending = False)

In [251]:
# df[['NAME','CHAMPIONS_KILLED']].groupby('NAME').sum()['CHAMPIONS_KILLED'].sort_values(ascending = False)

In [252]:
# temp = df.groupby('matchId').sum()[['NAME']]
# temp[temp['NAME'].str.contains('adostraa') != True].index

# df[(df['matchId'] == 4939492680) | (df['matchId'] == 4939565193)]['NAME']

In [253]:
# df.groupby('NAME').count()['CHAMPIONS_KILLED'].sort_values(ascending = False)

In [254]:
# def standardize (series):
#     return (series - series.mean())/series.std()

# temp_df = df
# standardize(temp_df[['NAME','TOTAL_DAMAGE_DEALT']].groupby('NAME').mean()).sort_values('TOTAL_DAMAGE_DEALT', ascending = False)


KeyError: "None of [Index(['NAME', 'TOTAL_DAMAGE_DEALT'], dtype='object')] are in the [columns]"

In [ ]:
grouped = df.groupby(['matchId', 'TEAM'])
teammate_counts = defaultdict(lambda: defaultdict(int))

for (match_id, team_number), group in grouped:
    players = group['NAME'].tolist()
    # Create all possible pairs of teammates
    for player1, player2 in combinations(players, 2):
        # Increment the count for each pair (both ways, since it's undirected)
        teammate_counts[player1][player2] += 1
        teammate_counts[player2][player1] += 1

# Convert the result to a more readable format (DataFrame)
teammate_stats = []
for player, teammates in teammate_counts.items():
    for teammate, count in teammates.items():
        teammate_stats.append({'player_name': player, 'teammate_name': teammate, 'count': count})

teammate_df = pd.DataFrame(teammate_stats)

In [ ]:
# name = 'TylerToTheT212'

# harb = teammate_df[teammate_df['player_name'] == name].sort_values('count',ascending = False)

# win_change = {
#     'Win' : True,
#     'Fail' : False
# }

# copy_df = df.copy()[['NAME','WIN']]

# copy_df['WIN'] = copy_df['WIN'].map(win_change)


# c = copy_df.groupby('NAME').mean().reset_index()

# c = c[c['NAME'].isin(np.array(harb['teammate_name']))]

# harb = pd.merge(harb,c, left_on = 'teammate_name', right_on='NAME')

# harb.drop(columns =['NAME'])
# harb['WITH_HARB'] = (harb['WIN'] * harb['count'])
# num = (harb['WIN'] * harb['count']).sum()/harb['count'].sum()
# print(num)
# harb

In [ ]:
role_df = df[['INDIVIDUAL_POSITION','NAME','matchId']].groupby(['NAME','INDIVIDUAL_POSITION']).count().reset_index()
role_df.sort_values('matchId',ascending=False).head(10)
role_df[role_df['NAME'] == 'DonutDude17']

,NAME,INDIVIDUAL_POSITION,matchId
17,DonutDude17,BOTTOM,4
18,DonutDude17,JUNGLE,11
19,DonutDude17,MIDDLE,1
20,DonutDude17,TOP,7
21,DonutDude17,UTILITY,15


In [ ]:
name = 'IbelieveMeYou'
champion = 'Poppy'

champ = df[['SKIN','NAME','WIN']]
champ['WIN'] = champ['WIN'].map(win_change)
champ = champ.groupby(['NAME','SKIN']).mean()
champ = champ


please = pd.merge(champ, df[['SKIN','NAME','matchId']].groupby(['NAME','SKIN']).count(), left_index=True,right_index= True)



please = please.reset_index()
please
please[please['NAME'] == name]

C:\Users\Zachary\AppData\Local\Temp\ipykernel_25356\3144391566.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champ['WIN'] = champ['WIN'].map(win_change)


,NAME,SKIN,WIN,matchId
137,IbelieveMeYou,Ashe,1.0,2
138,IbelieveMeYou,Belveth,0.0,2
139,IbelieveMeYou,Caitlyn,0.0,2
140,IbelieveMeYou,Chogath,0.0,1
141,IbelieveMeYou,Diana,0.666667,3
142,IbelieveMeYou,DrMundo,1.0,1
143,IbelieveMeYou,Ekko,1.0,1
144,IbelieveMeYou,FiddleSticks,0.0,2
145,IbelieveMeYou,JarvanIV,0.0,1
146,IbelieveMeYou,Jhin,1.0,1


In [ ]:
for col in df.columns:
    print(col)

matchId
gameDuration
gameVersion
ALL_IN_PINGS
ASSIST_ME_PINGS
ASSISTS
BAIT_PINGS
BARON_KILLS
BARRACKS_KILLED
BARRACKS_TAKEDOWNS
BASIC_PINGS
BOUNTY_LEVEL
CHAMPION_MISSION_STAT_0
CHAMPION_MISSION_STAT_1
CHAMPION_MISSION_STAT_2
CHAMPION_MISSION_STAT_3
CHAMPIONS_KILLED
CHAMPION_TRANSFORM
COMMAND_PINGS
CONSUMABLES_PURCHASED
DANGER_PINGS
DOUBLE_KILLS
DRAGON_KILLS
ENEMY_MISSING_PINGS
ENEMY_VISION_PINGS
EXP
FRIENDLY_DAMPEN_LOST
FRIENDLY_HQ_LOST
FRIENDLY_TURRET_LOST
GAME_ENDED_IN_EARLY_SURRENDER
GAME_ENDED_IN_SURRENDER
GET_BACK_PINGS
GOLD_EARNED
GOLD_SPENT
HOLD_PINGS
HORDE_KILLS
HQ_KILLED
HQ_TAKEDOWNS
ID
INDIVIDUAL_POSITION
ITEM0
ITEM1
ITEM2
ITEM3
ITEM4
ITEM5
ITEM6
ITEMS_PURCHASED
KEYSTONE_ID
KILLING_SPREES
LARGEST_ABILITY_DAMAGE
LARGEST_ATTACK_DAMAGE
LARGEST_CRITICAL_STRIKE
LARGEST_KILLING_SPREE
LARGEST_MULTI_KILL
LAST_TAKEDOWN_TIME
LEVEL
LONGEST_TIME_SPENT_LIVING
MAGIC_DAMAGE_DEALT_PLAYER
MAGIC_DAMAGE_DEALT_TO_CHAMPIONS
MAGIC_DAMAGE_TAKEN
MINIONS_KILLED
Missions_ChampionsKilled
Missions_Creep